# Introducció

En aquesta pràctica s'implementa un algoritme evolutiu basat en l'alrticle *Grigorios N. Beligiannis, Charalampos N. Moschopoulos, Georgios P. Kaperonis,Spiridon D. Likothanassis. Applying evolutionary computation to the school timetablingproblem: The Greek case. Computers & Operations Research 35 (2008) 1265 ? 1280*. L'objectiu es implementar l'algorisme com s'explica a l'article amb les mateixes entrades i sortides.

En aquest document s'explicara breument l'utilitat de cada funció. S'ha optat per introduïr tot el codi dins l'informe amb l'eina de programació *Jupyter Notebook*, que s'adjuntara juntament amb aquest document per a poder executar-ho. El codi de programació utilitzat és *Python*.

El codi està organitzat seguint el següent ordre: primer de tot s'inicialitzen les variables globals i es donen els valors corresponents a les variables que l'usuari ha de modificar per adaptar el programa a les seves dades. Seguidament es troben funcions que es criden al llarg del programa i, més envant, unes alres funcions que serveixen per calcular valors de cost relacionats amb els individus creats. Més envant es troba el codi encarregat de la creació de la primera població. Finalment, es troba el codi principal del programa format per un bucle que realitza iteracions sobre la població per a trobar nous individus millors i arribat a trobar el millor horari segons la funció de cost.

Al final del document s'exposa un apartat final de conclusions i comentaris respecte a la implementació d'aquesta pràctica, explicant els problemes que s'han anat trobant i les millores s'haurien de fer.

# Implementació

## Inicialitzacions

Primer de tot podem observar les llibreries importades per poder implementar l'algorisme. Principalment es fa ús de *numpy*, una llibreria que permet operacions matemàtiques bàsiques i la creació de variables de cert tipus. Seguidament es fa servir la llibreria *random* per poder crear valors aleatoris com, per exemple, fer la selecció d'un individu per a que passi a la següent generació. A part d'aquestes llibreries també es fan servir *copy* y *operator* per a altres funciones especials.

In [ ]:
#import pandas as pd
import numpy as np
from random import randrange
import copy
import random
import operator

Una vegada importades les llibreries necessaries, s'han d'introduïr les dades per a la creació d'un horari que, en aquest cas, correspon a l'horari dels professors d'una escola. Algunes de les variables importants a mencionar son:

**classes:** Nombre de classes que es volen incertar (més envant).

**population:** Tamany de la població, és a dir, quants d'individus es volen dins cada població generada per l'algorisme.

**TeachingHours:** Hores de que cada porfessor ha d'exercir durant la setmana.

**TeacherNoDays:** Dies que els professors no poden assistir a classe.

**alg_iterations:** Nombre d'iteracions a realitzar.


In [ ]:
#Inicialitzacions
classes=2
TimeP=35
Chromo=[]
Chromo=[[[0,0] for i in range(TimeP)] for i in range(classes)]
population=25
TeachingHours=np.array([11,6,7,10,9,5])
ZerosT=np.zeros((len(TeachingHours),),dtype=int)
ZerosT=TeachingHours*0
numTeachers=len(TeachingHours)
Thours=np.zeros((classes, numTeachers),dtype=int)
AssigsProfe=[[[] for i in range(numTeachers)] for i in range(classes)]
Thours=Thours.tolist()
print(Thours)
#0->Profe a l'institut, 1-> profe no és a l'institut
TeacherNoDays=[[1,0,0,0,0],[0,1,0,0,0],[0,0,0,1,1],[0,0,0,0,0],[0,0,1,0,0],
               [0,0,0,0,0]]
alg_iterations = 10000


## Entrada de Dades Principals

A continuació s'introdueixen les dades principals de cada professor. Més concretament, la variable *ClasseAssignatura* és una llista de llistes que conté la informació necessaria de cada assignatura. A continuació es veu un exemple, on cada assignatura conté quatre nombres. El primer nombre indica les hores que s'han d'impartir a la setmana d'aquesta assignatura; el segon indica el professor principal d'aquesta assignatura; el tercer indica amb quina assignatura es fa *subclasse* (0 en cas de no haver *subclasse*); i finalment, el quart nombre indica amb quin professor s'imparteix *co-teaching* (0 de nou s no es fa *co-teaching*).vCada llista definida dins d'aquesta variable *ClasseAssignatura* correspondrà a una classe diferent.

In [ ]:
ClasseAssignatura=[[[3,1,0,0],[2,2,0,0],[4,3,0,0],[3,1,0,0],[3,4,5,0],
                     [3,4,0,7],[3,6,0,6]],
                    [[3,1,0,0],[4,2,0,4],[3,3,0,0],[4,4,0,2],[6,5,0,0],
                     [2,6,0,0],[2,1,0,0]]]
print(ClasseAssignatura)
ClasseAssignatura2 = copy.deepcopy(ClasseAssignatura)

## Funcions Auxiliars

### Calculate_TeachingHours()

Aquesta funció té com a objectiu calcular les hores que ha de donar cada professor a cada classe. Aquest resultat es calcula a partir de la informació emmagatzemada a *ClasseAssignatura*. Aquesta funció s'utilitza bàsicament per saber les hores de cada professor durant la creació de la primera població.

In [ ]:
def Calculate_TeachingHours():    
    c=0
    for classe in ClasseAssignatura:

        for assignatura in classe:
            professor=assignatura[1]
            hores=assignatura[0]
            #Per casa assignatura sumam en 1 el num d'assignatures al profe que la dona
            #Només calculam les hores de profe "principal"
            #Aquest valor l'utilitzarem al cost de les classes
            Thours[c][professor-1]=Thours[c][professor-1]+hores
            #Si hi ha SC:
            if assignatura[2]!=0:
                profeSC=assignatura[2]
                Thours[c][profeSC-1]=Thours[c][profeSC-1]+hores  

        c=c+1;
#     print(Thours)    

### getDictProfes()

Aquesta funció s'ha creat per poder relacionar, d'alguna manera, les assignatures amb els professors. Més concretament, per poder guardar les combinacións úniques de professors que poden donar una assignatura en concret. Això és important per poder calcular el cost d'un individu. S'intenta crear una estructura de tipus diccionari (clau - valor), ja que es necessita una data de tipus llista (o tupla) dins de cada clau pero un diccionar normal de python no ho permet.

Dins de cada valor corresponent a cada clau s'indiquen tres valors: el primer valor (*0*) fa referència a l'index de l'assignatura que dona el professor (o combinació de professors) lligat a la clau; el segon valor (*1*) indica el nombre d'hores que s'han d'impartir per cada assignatura; el tercer i darrer valor (*2*) es fa servir per indicar si aquesta assignatura ja s'ha utilitzat per completar un periode dins un mateix dia. Aquest darrer valor es fa servir per crear els horaris de les classes i, posteriorment, calcular els valors de cost.

In [ ]:
#idx,numhores,utilitzat

def getDictProfes():
    dctKTeach=[]
    dctVTeach=[]
    for classe in ClasseAssignatura:
        idx_Assig=0
        ProfesK=[]
        Profesv=[]
        for assignatura in classe:
            #Clau del diccionari profes 
            profes=[0,0]
            profes[0]=assignatura[1]
            if assignatura[2]!=0:
                profes[1]=assignatura[2]
            if assignatura[3]!=0:
                ProfeCT=classe[assignatura[3]-1][1]
                profes[1]=ProfeCT 
            # Quan hi hagi coteaching posam només el numero de una de les
            # assignatures, l'altra està assignada de manera implicita
            
            # Contingut a afegir (dictvalue)
            numhores=assignatura[0]
            # [Numassignatura, hores, utilitzat]
            infoAssig=[idx_Assig+1,numhores,0]
            
            if profes in ProfesK:
                Profesv[ProfesK.index(profes)].append(infoAssig)
            else:
                ProfesK.append(profes)
                Profesv.append([infoAssig])
                

            idx_Assig=idx_Assig+1
        dctKTeach.append(ProfesK)
        dctVTeach.append(Profesv)
        
    return dctKTeach,dctVTeach

In [ ]:
Thours=np.zeros((classes, numTeachers),dtype=int)
AssigsProfe=[[[] for i in range(numTeachers)] for i in range(classes)]
Calculate_TeachingHours()

d_TeachersK,d_TeachersV=getDictProfes()
classTimeT=[0 for i in range(population)]

### IdealTeachingDayHours()

Aquesta funció calcula la millor repartició d'hores setmanals per un professor tenguent en compte el nombre d'hores que ha d'impartir i els dies que assisteix a l'escola. Per exemple, si un professor ha d'impartir 11 hores de classe i assisteix 4 dies a la setmana, el nombre d'hores ideals per dia serien 11/4 = 2.75. Aquest valor s'utilitza per calcular valors de cost.

In [ ]:
def IdealTeachingDayHours():
    TeacherIdealHours=[]
    for t in range (numTeachers):
        horesideals=TeachingHours[t]/(5-np.count_nonzero(TeacherNoDays[t]))
        TeacherIdealHours.append(horesideals)
    return TeacherIdealHours

In [ ]:
TeacherIdealHours=IdealTeachingDayHours()
print(TeacherIdealHours)

### decrementarHores()

A l'hora de crear la primera població, aquesta funció, com el propi nom indica, s'encarrega de decrementar les hores dels professors corresponents, ja que quan es selecciona aleatoriament un porfessor per crear la primera població és necessari saber quantes hores disponibles li queden per completar.

In [ ]:
#Recorrem les assignatures d'una classe
#[slots,professor1,professorSC,AssignaturaCT]

def decrementarHores(teacher,classe,Thours):
    #Recorrem les assignatures de la classe buscant les que fa el profe teacher
    #c= assignatures de la classe 1
    
    idxAssigntr=0

    c=ClasseAssignatura2[classe][:]

    for assignatura in c:

        if assignatura[1]==teacher and assignatura[0]>0:

            #Decrementam 
            ClasseAssignatura2[classe][idxAssigntr][0]=ClasseAssignatura2[classe][idxAssigntr][0]-1
            Thours[classe][teacher-1]=Thours[classe][teacher-1]-1
            teachers=[teacher,0]
          
            
            #si hi ha SC les decrementam de Thours
            if ClasseAssignatura2[classe][idxAssigntr][2]!=0:
                
                teacher2=ClasseAssignatura2[classe][idxAssigntr][2]
               
                Thours[classe][teacher2-1]=Thours[classe][teacher2-1]-1
                teachers=[teacher,teacher2]
               
                
            #Si hi ha coteaching llevam les hores de l'altra assignatura:
            if ClasseAssignatura2[classe][idxAssigntr][3]!=0:
                #Seleccionam el valor de l'index de l'assignatura amb la que hi ha coteaching
                AssigCT=ClasseAssignatura2[classe][idxAssigntr][3]
                
                #Decrementam en 1h les hores pendents d'assignar de l'assignatura de CT
                ClasseAssignatura2[classe][AssigCT-1][0]=ClasseAssignatura2[classe][AssigCT-1][0]-1
                teacherCT=ClasseAssignatura2[classe][AssigCT-1][1]
                Thours[classe][teacherCT-1]=Thours[classe][teacherCT-1]-1
                teachers=[teacher,teacherCT]
                
            
            break
                
        idxAssigntr=idxAssigntr+1
    #c=ClasseAssignatura2[classe][:]
    
    #print("ClasseAssignatura2 decrem",ClasseAssignatura2)
    return Thours,teachers
                
        

### mutation()

Aquesta funció és una de les principals del programa i s'encarrega de mutar una població. Per a implementar aquesta funció s'ha seguit l'explicació de l'article. Per una banda s'implementa la *period mutation* i, per altra, la *bad_mutation*. Però, com explicarem a l'apartat de conclusions, la *bad mutation* no s'ha conseguit implementar sense errors. El funcionament bàsic de la mutació *period mutation* és triar aleatoriament dues hores dins una classe i dins un individu i girar les seves posicions entre elles. Aquesta mutació només es realitzarà si un número aleatori és menor a una probabilidad indicada com *period_mutation_probability*.

In [ ]:
def mutation(pob, ctp):
    period_mutation_probability = 0.05
    bad_mutation_probability = 0.05

    pob_mutada=copy.deepcopy(pob)
    pob_final=copy.deepcopy(pob)

#     black_list = []
    # Botam el primer individu per no modificar la millor opcio
    for i_individu in range(1,len(pob_mutada)):
        
        for i_classe in range(len(pob_mutada[i_individu])):
            
            # ------------------------------------------------Period mutation
            if random.random() < period_mutation_probability:
                
#                 tp_ok = False
#                 while tp_ok == False:
#                     random_tp1 = random.randrange(0, len(pob[i_individu][i_classe]))
#                     random_tp2 = random.randrange(0, len(pob[i_individu][i_classe]))
#                     if (random_tp1 not in black_list) & (random_tp2 not in black_list):
#                         tp_ok = True
#                         black_list.append(random_tp1)
#                         black_list.append(random_tp2)
                        
                random_tp1 = random.randrange(0, len(pob[i_individu][i_classe]))
                random_tp2 = random.randrange(0, len(pob[i_individu][i_classe]))

                tp_1 = pob_mutada[i_individu][i_classe][random_tp1]
                tp_2 = pob_mutada[i_individu][i_classe][random_tp2]
        
                pob_mutada[i_individu][i_classe][random_tp1] = tp_2
                pob_mutada[i_individu][i_classe][random_tp2] = tp_1

#                 print("Individu mutat: ",i_individu)
#                 print("--- CLASSE MUTADA ---", i_classe)
#                 print("Periode 1 ", random_tp1)
#                 print("Periode 2 ", random_tp2)

            
            # ------------------------------------------------Bad mutation
            m_tp1_total = 0
            m_tp2_total = 0
            found_tp = False
            
            if random.random() < bad_mutation_probability:
                while found_tp == False:
                    for t in range (len(ctp[i_individu])):
                        #Cada profesor t
                        m_tp1 = max(ctp[i_individu][t][i_classe])
                        if m_tp1 > m_tp1_total:
                            m_tp1_total = m_tp1
                            idx_tp1 = ctp[i_individu][t][i_classe].index(m_tp1)
                            idx_t_tp1 = t
                            print("INDEX MAX 1(t:{},c:{},idx:{}): {}".format(idx_t_tp1, i_classe,idx_tp1, m_tp1))
                            
                    # Per poder treure el segon màxim
                    ctp[i_individu][idx_t_tp1][i_classe][idx_tp1] = 0
                            
                    for t in range (len(ctp[i_individu])):
                        #Cada profesor t
                        m_tp2 = max(ctp[i_individu][t][i_classe])
                        if m_tp2 > m_tp2_total:
                            m_tp2_total = m_tp2
                            idx_tp2 = ctp[i_individu][t][i_classe].index(m_tp2)
                            idx_t_tp2 = t
                            print("INDEX MAX 2(t:{},c:{},idx:{}): {}".format(idx_t_tp2, i_classe, idx_tp2, m_tp2))
                        
                    
                    
                    # Comprobar TeacherNoDays
                    dia_t1 = int(idx_tp1/7)
                    dia_t2 = int(idx_tp2/7)
                    print("Dies: ", dia_t1, dia_t2)
                    print("Teacher no days: ", TeacherNoDays[idx_t_tp1], TeacherNoDays[idx_t_tp2])
                    if (TeacherNoDays[idx_t_tp1][dia_t1] == 0) & (TeacherNoDays[idx_t_tp2][dia_t2] == 0):
                        found_tp = True
                    else:
                        ctp[i_individu][idx_t_tp2][i_classe][idx_tp2] = 0
                        m_tp1_total = 0
                        m_tp2_total = 0
                
                tp_1 = pob_mutada[i_individu][i_classe][idx_tp1]
                tp_2 = pob_mutada[i_individu][i_classe][idx_tp2]
        
                pob_mutada[i_individu][i_classe][idx_tp1] = tp_2
                pob_mutada[i_individu][i_classe][idx_tp2] = tp_1
                        
                
                
                
#     for i in range(len(pob)):
#         print("individu",i)
#         print("ORIGINAL", pob[i][0])
#         print(pob[i][1])
#         print("MUTAT   ", pob_mutada[i][0])
#         print(pob_mutada[i][1])
    
#     print(individusHanMutat)
#     print("P_ORI", pob)
#     print("P_MUT", pob_mutada)
#    print("Son iguals??????? ",pob_mutada==pob)

    return pob_mutada

## Funcions de Cost

A partr d'aqui es mostren les funcions necessaries que s'han hagut d'implementar per calcular els valors de cost. Totes aquestes funcions individuals s'executen després per dues funcions de cost principals, *calc_C1_C2()* i *CostTeacherE_i_D()*, que finalment son executades a la funció de cost genèrica *costFunction()*

A continuació es mostren variables que representen els valors dels paràmetres que s'utilitzen per calcular els valors dels costs. S'indiquen amb un comentari els valors indicats per l'article.

In [ ]:
BASE=1.5  # 1.5
HCW=10    # 10
IDWT=3    # 3
TEPW=2    # 2


### CTeacherUnable()

Respecte als costs definits a l'article, aquesta funció s'encarrega de calcular el cost relacionat amb la no disponibilitat del professor en un dia en concret. Es a dir, si a un horari definit per un individu hi ha assignat un professor a un dia que ell no hi assisteix, el cost ha d'augmentar seguint l'expressió definida a l'article.

In [ ]:
def CTeacherUnable(teachers, hora, classe_ind, individu, costsHoresProfes):

    dia=int(hora/7)
    #print("dia",dia)
    cost=0
    #print("profes",teachers)
    
    for teacher in teachers:
        if teacher != 0: 
            #print("at School?")
            #print(teacher,dia)
            #print(TeacherNoDays[teacher-1][dia])
            #print("classe_ind",classe_ind)
            c=TeacherNoDays[teacher-1][dia]*(HCW*(BASE**3))
            costsHoresProfes[teacher-1][classe_ind][hora]=costsHoresProfes[teacher-1][classe_ind][hora]+c
            
            cost=cost+TeacherNoDays[teacher-1][dia]
            #costsTeachers[individu][teacher-1]=costsTeachers[individu][teacher-1]+(HCW*(BASE**3))
    #cost = 0
    costsHoresProfe = 0
    return cost,costsHoresProfes

### C_classEmptySpaces()

Aquesta funció fa referencia al cost relacionat amb els espais buits dins la classe que es troben sense assignar cap professor. Per cada un d'aquests buits s'incrementa el cost basant-se amb l'expressió indicada a l'article.

In [ ]:
#HCW*BASE^BASE
def C_classEmptySpaces(individu):
    costClEmptySpaces=0
    emptyHour=False
    
    for classe in individu:
        timeP=0
        #cLessonDispersion=costClLessDispersion(classe)
        for teachers in classe:
            #Si canviam de dia resetejam les variables          
            if timeP%7==0:
                emptyHour=False
            #Si hi ha una hora buida activam empty hour  
            if teachers==[0,0]:
                emptyHour=True
            #Si hi ha una hora ocupada i prèviament hi ha hagut una hora buida el mateix dia sumam 1 al cost
            elif emptyHour==True:
                emptyHour=False
                costClEmptySpaces=costClEmptySpaces+1
            timeP=timeP+1
    cost=costClEmptySpaces*HCW*(BASE**BASE)
    return cost  

### createClassTimetable()

Aquesta funció s'encarrega de crear un horari per una classe donada, és a dir, indica quina assignatura es dona a cada hora de la setmana. Per poder fer-ho, utilitza com a entrada el diccionari creat anteriorment amb la funció *getDictProfes()*.

A part de crear aquest horari, també s'encarrega de calcular el cost relacionat amb la disperssió d'hores per als professors. Es a dir, cada hora diferent a les hores ideals calculades anteriorment, sumen un cost adicional al cost final de l'individu.

In [ ]:
#Crea els horaris per cada classe i calcula lesson dispersion.
def createClassTimetable(individu):
    classTimeT=[[0 for i in range(TimeP)] for i in range(classes)]
    
    dtKeys=copy.deepcopy(d_TeachersK)
    dtValues=copy.deepcopy(d_TeachersV)
    i_class=0
    costClLessonDisp=0
    #print(dtKeys)
    #print(dtValues)
    for classe in individu:
        h=0
        c_hours=0
        c_days=0
        h2=0
        for day in range(5):
            for hour in range(7):
                teachers=individu[i_class][h]
                if teachers==[0,0]:
                    classTimeT[i_class][h]=0
                else:
                    indx=dtKeys[i_class].index(teachers)
                    
                    #Buscam la assignatura d'aquest profe/s amb valor d 'utilitzat menor
                    #que encara tengui hores per assignar
                    AssigProfe=copy.deepcopy(dtValues[i_class][indx])
                    
                    #Ordenam de menor a major segons el valor d'utilitzat:
                    AssigProfe.sort(key=lambda x: x[2])
                    
                    #Assignam una hora d'aquesta assignatura i la marcam com utilitzada
                    #Buscam una assignatura a la que encara li quedin hores
                    i=0
                    while AssigProfe[i][1]==0:
                        i=i+1
                        
                    #Guardam numAssignatura
                    classTimeT[i_class][h]=AssigProfe[i][0]
                    i_assig=dtValues[i_class][indx].index(AssigProfe[i])
                    
                    #Incrementam utilitzat
                    dtValues[i_class][indx][i_assig][2]=dtValues[i_class][indx][i_assig][2]+1
                    #idx,numhores,utilitzat
                    #Decrementam les hores 
                    dtValues[i_class][indx][i_assig][1]=dtValues[i_class][indx][i_assig][1]-1
                h=h+1
            #Canvi de dia:
            
            DaySubjects=classTimeT[i_class][h2:h2+7]
            DaySubjects=[elem for elem in DaySubjects if elem!=0]
            NumRepeatedS=len(DaySubjects)-len(set(DaySubjects))
            c_hours=c_hours+NumRepeatedS
            if NumRepeatedS >0:
                c_days=c_days+1
            h2=h2+7
           
            #Reset de utilitzat:
            t=0
            for AssigT in dtValues[i_class]:
                for assig in range(len(AssigT)):
                    dtValues[i_class][t][assig][2]=0
                t=t+1 
        #Calculam el cost del dia  IDWC ∗ HOURS ∗ BASE^DAYS:
        costClLessonDisp=costClLessonDisp+IDWT*c_hours*(BASE**c_days)
        i_class=i_class+1
    return classTimeT,costClLessonDisp

### createTeachersTimeP()

De la mateixa manera que la funció anterior, aquesta funció s'encarrega de crear un horari però, en aquest cas, per als professors, el qual es fa servir més envant per calulcar els costs.

In [ ]:
#Funció per crear l'horari de cada un dels profes per cada individu.

def createTeachersTimeP(teachers,i_classe,i_timePeriod,TeachersTimeTable):
    #print(teachers)
    for teacher in teachers:
        if teacher!=0:
            #Guardam la classe que fa aquest profe en aquesta hora
            TeachersTimeTable[teacher-1][i_timePeriod]=i_classe+1
            #TeachersTimeTable[teacher-1][i_timePeriod].append(i_classe+1)
    return TeachersTimeTable


### calc_C1_C2()

Aquesta funció, juntament amb la següent (*CostTeacherE_i_D*), conformen les dues parts principals per calcular els costs. S'han creat separades perque cada una recorr les dades d'una manera més apropiada per al tipus de dades que esta tratant. Aquestes funcions van cridant les funcions individuals de cost explicades anteriorment i després son executades a la funció de cost principal *costFunction()*.

In [ ]:
def calc_C1_C2(individu,indx_individu,costsHoresProfes):
    costUnable=0
    costMore1class=0
    TeachersTimeTable=[[0 for i in range(TimeP)] for i in range(numTeachers)]
    hours=[]
    llista_classes=[]
    #llista dels profes assignats a una hora
    teacher_timeP=[]
    #for individu in population:
    for i_timePeriod in range(TimeP):
        teacher_timeP=[]
        for i_classe in range(classes):
            teachers=individu[i_classe][i_timePeriod]
            #print("profes",teachers)
            
            #Cream els horaris dels profes.
            TeachersTimeTable=createTeachersTimeP(teachers,i_classe,i_timePeriod,TeachersTimeTable)
            
            #Calculam el cost de si el profe no és a l'institut
            costU,costsHoresProfes=CTeacherUnable(teachers,i_timePeriod,i_classe,indx_individu,costsHoresProfes)
            #if costU!=0:
                #print("--------------------")
                #print(i_timePeriod)
                #print("teachers",teachers)
                #print("costU",costU)
                #print("costUnable",costUnable)

            for teach in teachers:
                if teach!=0:
                    teacher_timeP.append(teach)
                    hours.append(i_timePeriod)
                    llista_classes.append(i_classe)
            
            costUnable=costUnable+costU
            #print("cost, i_timePeriod",costU,i_timePeriod)

        #Eliminam de la llista els buits(teacher 0) ja que teacher 0 no és cap teacher
        #teacher_timeP=[elem for elem in teacher_timeP if elem != 0]
        #print(set(teacher_timeP))
        #print(teacher_timeP)
        cM1c_K=len(teacher_timeP)-len(set(teacher_timeP))
        costMore1class=costMore1class+HCW*(BASE**cM1c_K)
        #Calculate the cost for each teacher
        h3=0
        for t in teacher_timeP:
            k=teacher_timeP.count(t)-1
            teachersM1c_Cost=HCW*(BASE**k)
            costsHoresProfes[t-1][llista_classes[h3]][hours[h3]]=costsHoresProfes[t-1][llista_classes[h3]][hours[h3]]+teachersM1c_Cost
            #costsTeachers[indx_individu][teacher-1]=costsTeachers[indx_individu][teacher-1]+HCW*(BASE**k)
            h3=h3+1

    costUnable=costUnable*(HCW*(BASE**3))
    return(costUnable,costMore1class,TeachersTimeTable,costsHoresProfes)


In [ ]:
#Cost Teacher empty spaces and dispersion
def CostTeacherE_i_D(idx_individu,TeachersTimeTable1,costsHoresProfes):
    costTeacherEmpty=0
    costTeacherLDisp=0
    cTeacherDisph=0
    cTeacherDispd=0
    hoursTeacher=0#num hores que fa un profe cada dia
    TotalcostTeacherDisp=0
    EmptyFound=False
    TeacherIdealHours=IdealTeachingDayHours()
    t=0#index del professor
    
    for teacher in TeachersTimeTable1:
        h=0
        chours=0#cost hores
        cdays=0 #cost dies
        for day in range(5):
            incr=False
            thr=1
            for hour in range(7):
                
                if teacher[h]!=0:
                    hoursTeacher=hoursTeacher+1
                    if abs(hoursTeacher-TeacherIdealHours[t])>=thr:
                        clP_idx = teacher[h] - 1
                        costsHoresProfes[t][clP_idx][h]=costsHoresProfes[t][clP_idx][h]+IDWT*BASE
                        thr=thr+1
                        
                    #Si trobam una hora ocupada després de trobar un espai buit
                    if EmptyFound==True:
                        costsHoresProfes[t][clP_idx][h]=costsHoresProfes[t][clP_idx][h]+TEPW*BASE
                        chours=chours+1
                        incr=True
                        EmptyFound=False
                    
                elif teacher[h]==0:
                    EmptyFound=True
                    
                h=h+1#index de les hores
                
            #Quan canviam de dia incrementam el cost del dia i resetejam els paràmetres:
            #Si les hores que fa un professor un dia es desvien en més de 1 de la mitjana ideal:
            hour_difference=abs(hoursTeacher-TeacherIdealHours[t])
            if hour_difference >= 1:
                cTeacherDisph=cTeacherDisph+hour_difference
                cTeacherDispd=cTeacherDispd+1
            if incr==True:
                cdays=cdays+1
            hoursTeacher=0
            incr=False
            EmptyFound=False
            
        #Càlcul del cost de lesson's dispersion
        costTLD=IDWT*cTeacherDisph*(BASE**cTeacherDispd)
        costTeacherLDisp=costTeacherLDisp+costTLD
        #costsTeachers[idx_individu][t-1]=costsTeachers[idx_individu][t-1]+costTLD
        
        #Aquí calcul del cost de Empty Scpaces
        costES=TEPW*chours*(BASE**cdays)
        costTeacherEmpty=costTeacherEmpty+costES
        #costsTeachers[idx_individu][t-1]=costsTeachers[idx_individu][t-1]+costES
        
        t=t+1
        cTeacherDisph=0
        cTeacherDispd=0
        
    return costTeacherLDisp,costTeacherEmpty,costsHoresProfes
            

In [ ]:
#costs timeP -> Per cada teacher es crea una llista amb els costs dels 35 timePeriods.
costsHoresProfes=[[0 for i in range(35)] for i in range(numTeachers)]
cTeacherTimeP=[]
cTeacherTimeP.append(costsHoresProfes)
cTeacherTimeP.append(costsHoresProfes)
# print(cTeacherTimeP[0])

### costFunction()

Aquesta és la funció que engloba tots els costs calcualts amb les funcions anteriors. Per paràmetre s'introdueix la població de la qual es vol calcular el cost i es retorna el cost de cada individu.

Notar que, fins aquest punt, en cap moment s'ha fet menció del cost indicat a l'article *wrong co-teaching*. Això és degut a que, per la manera que s'ha implementat el programa, no pot provocar-se cap situació on es generi un co-teaching erroni.

In [ ]:
def costFunction(pobl):    
    #costsTeachers=[[0 for i in range(numTeachers)] for i in range(population)] #declarar antes de cridar funcio
    costs=[0 for i in range(len(pobl))]
    costsHoresProfes=[[[0 for i in range(TimeP)] for i in range(classes)] for i in range(numTeachers)]
    #print("costsHoresProfes", costsHoresProfes)
    cTeacherTimeP=[]
    costsTimeP=[]
    idx_individu=0
    
    
    for individu in pobl:

        cTU,cM1cl,TeachersTimeTable,costsHoresProfes=calc_C1_C2(individu,idx_individu,costsHoresProfes)
        cES=C_classEmptySpaces(individu)
        cTE,cTLD,costsHoresProfes=CostTeacherE_i_D(idx_individu,TeachersTimeTable,costsHoresProfes)
        classTimeT[idx_individu],cClassLDisp=createClassTimetable(individu)
#         print(classTimeT)
#         print("cTU ",cTU)
#         print("cM1cl ",cM1cl)
#         print("cES",cES)
#         print("cTE ",cTE)
#         print("cTLD ",cTLD)
#         print("cClassLDisp ",cClassLDisp)

        TotalCost=cTU+cM1cl+cES+cTE+cTLD+cClassLDisp
        costs[idx_individu] = TotalCost
        idx_individu=idx_individu+1
        copia=copy.deepcopy(costsHoresProfes)
        cTeacherTimeP.append(copia)
    return costs,cTeacherTimeP

## Creació de la primera població

Una vegada s'han introduït les dades i s'executa el programa, la primera passa que es segueix és la creació de la primera població. Mitjançant les funcions auxiliars mencionades anteriorment, es van seleccionant professors aleatoriament i, si encara tenen hores per impartir disponibles, son seleccionats per a omplir el pròxim periode (hora). La funció creará tants d'individus com l'usuari hagia assignat a la variable *population* i, una vegada generats, l'algorisme ja esta llest per fer un bucle a on es seleccionaran i mutaran individus per a les següents poblacions, és a dir, les noves generacions.

In [ ]:
poblacio=[0 for i in range(population)]
for i in range(population):
    Thours=np.zeros((classes, numTeachers),dtype=int)
    Calculate_TeachingHours()
    ClasseAssignatura2=copy.deepcopy(ClasseAssignatura)
    for classe in range(classes):
        for hour in range(TimeP):
            c=copy.deepcopy(ClasseAssignatura[classe])
            ok=False
            #Chose a teacher at random 
#             print("classe= ",classe)
#             print(Thours[classe])
            
            # Miram si s'han assignat totes les hores d'una classe.
            if np.array_equal(ZerosT,Thours[classe])==False:
                while ok==False:#mirar que no sigui que no et quedin profes i se quedi aqui infinitament
                    teacher=randrange(1, numTeachers+1)
                    for j in c:
                        #print("i",i) 
                        #print("c",c)
                        #print("i[1]",i[1])
                        #print(Thours[classe][teacher-1])
                        if(j[1]==teacher) and Thours[classe][teacher-1]!=0:
                            ok=True
                    #ok=True       

                Thours,teachers=decrementarHores(teacher,classe,Thours)
                Chromo[classe][hour]=teachers
#                 print("TeacherHours, teachers :",Thours[classe], teachers)
                
            
                #Assignar slot buit
#             print("buit")
#     print("index",i)      
    poblacio[i]=copy.deepcopy(Chromo)
 

## Loop iteratiu

Una vegada generada la primera població, la seguent passa és anar generant noves poblacions on el cost d'elles vagi disminuint a mesura que es van executant les iteracions. 

Primer de tot i com s'ha dit anteriorment, es fa una selecció dels individus de la població actual per a que formin part de la següent. Per a fer-ho, es defineix un vector dividit en segments, on cada segment representa la probabilitat de reproducció de cada individu depenent de la grandària de l'interval al que esta assignat. A continuació es mostra com s'ha generat aquest vector de probabilitats que va de 0 a 1 i es guarda dins la variable *mapped*.

In [ ]:
N = population
n_min = 0.2
n_max = 2 - n_min
    
probability = []

# Càlcul de probabilitat de reproduccio
for i in range(N):
    probability.append((1/N)*(n_min+(n_max - n_min)*(i)/(N-1)))
    
# Vector de probabilitats de reproduccio
mapped = []
mapped.append(0)

for i in range(len(probability)-1):
    mapped.append(mapped[i] + probability[i])

mapped.append(1)

A la següen cel·la trobam el bucle principal, format per l'estructura for: *for i in range(alg_iteracions):*. Com s'ha comentat abans, primer es fa la selecció i després la mutació. Abans de fer la selecció es calculen els costs de la població actual i, un cop calculats, s'ordenen els individus en funció dels costs calculats de manera descendent. Això es fa mitjançant un diccionari, ja que necessitam saber tant l'index de l'individu (clau) com el seu valor del cost (valor). Un cop ordenats, es seleccionen mitjançant un valor aleatori i les probabilitats calculades a la cel·la anterior.

Un cop realitzada la selecció es crida la funció de mutació sobre la nova població intermitja. Això provocara que alguns individus aleatoris sofreixin canvis dins la seva configuració, la qual pot ser positiva o negativa, i mitjançant la funció de cost es sabrà si el canvi ha estat positiu o negatiu.

Una passa important en aquest bucle és la inserció del millor cromosoma de la població actual a la població de la nova generació per tal de mantenir el millor individu calculat fins al moment. Això es fa, un pic ordenats els individus abans de la selecció, es guarda el valor del darrer individu, ja que es troben ordenats de manera descendent en funció del cost.

In [ ]:
cost_iteracio = list(range(alg_iterations))
# aux = 0

for i in range(alg_iterations):
    print("===============================================================[{}]".format(i))

    costsTeachers=[[0 for i1 in range(numTeachers)] for i2 in range(population)]
    calculatedCosts,x = costFunction(poblacio)
    
    cost_iter = copy.deepcopy(calculatedCosts)
    cost_iteracio[i] = cost_iter[0]
    
    #print("costs", calculatedCosts)
    
    #-------------------------------------------------------- SELECCIÓ
    # Creació d'un diccionari per ordenar els individus per cost.
    costs={}
    for i_p in range(population):
        costs[i_p]=calculatedCosts[i_p]
    #print(costs)

    sorted_costs = sorted(costs.items(), key=operator.itemgetter(1))
    sorted_costs.reverse()
#     print(sorted_costs)
    
    # Creació de la nova poblacio   
    nova_poblacio = copy.deepcopy(poblacio)

    # Elitism: aseguram que el millor cromosoma pasa a la següent generacio
    nova_poblacio[0] = poblacio[sorted_costs[-1][0]]

    # Selecció dels individus a reproduir
    for j in range(N-1):            
        rand = random.random()
        for k in range(len(probability)):
            if (rand > mapped[k]) & (rand <= mapped[k+1]):
                cromo_index = k
                break
        # Aqui s'ha triat el cromosoma que s'ha de reproduir. Ara l'hem de reproduir
        
        #nova_poblacio.append(poblacio[sorted_costs[cromo_index][0]])
        nova_poblacio[j] = poblacio[sorted_costs[cromo_index][0]]

    nova_poblacio[0] = poblacio[sorted_costs[-1][0]]
    elitism = copy.deepcopy(poblacio[sorted_costs[-1][0]])
    
    
    #-------------------------------------------------------- MUTACIÓ
    poblacio_a_mutar = copy.deepcopy(nova_poblacio)
    
    costsTeachers=[[0 for i1 in range(numTeachers)] for i2 in range(population)]    
    calculatedCosts,ctp_am = costFunction(poblacio_a_mutar)
    poblacio_mutada = mutation(poblacio_a_mutar, ctp_am)

#     for asdf in range(len(poblacio)):
#         print("-->INDIVIDU", asdf)
#         print("ORIGINAL", poblacio[asdf])
#         print("MUTADA  ", poblacio_mutada[asdf])
    
    poblacio = copy.deepcopy(poblacio_mutada)
    
    # Elitism: aseguram que el millor cromosoma pasa a la següent generacio
    poblacio[0] = elitism
    
    

En aquesta part es representa el cost de l'individu més bo que s'ha trobat a cada iteració. Com es pot observar, a mesura que es va executant l'algorisme, el cost mínim va baixant.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

l = list(range(alg_iterations))
plt.plot(l, cost_iteracio)
plt.show()

print(cost_iteracio[-1])

*Teacher unavailability: For each teacher and for each time period this teacher is assigned to a class, while he/she is unavailable at school the corresponding day, the following cost is added: HCW ∗ BASE3.

*Teacher assigned in more than one class: For each teacher and for each time period, if this teacher is assigned to more than one class, for example, to k classes, the following cost is added: HCW ∗ BASEk


Teacher lessons’ dispersion: For each teacher the following subcost is added: IDWT ∗ HOURS ∗ BASEDAYS, where
HOURS is the total number of time periods that are different compared to the ideal dispersion teacher’s timetable
and DAYS is the number of days in the teacher’s timetable that such a declination from the ideal case exists.